import necessary libraries

In [1]:
import numpy as np
import pickle


data_path = '../Data/training_data/period0/'

### The model we'll be using below

In [2]:
import numpy as np
import torch as t
import torch.nn as nn

from torch import LongTensor as LT
from torch import FloatTensor as FT

 
class Bundler(nn.Module):

    def forward(self, data):
        raise NotImplementedError

    def forward_i(self, data):
        raise NotImplementedError

    def forward_o(self, data):
        raise NotImplementedError


class Word2Vec(Bundler):

    def __init__(self, vocab_size=20000, embedding_size=300, padding_idx=0):
        # this instantiate the parent class
        super(Word2Vec, self).__init__()
        self.vocab_size = vocab_size
        self.embedding_size = embedding_size
        # define embeddings for the target word matrix and the context word matrix
        self.ivectors = nn.Embedding(self.vocab_size, self.embedding_size, padding_idx=padding_idx)
        self.ovectors = nn.Embedding(self.vocab_size, self.embedding_size, padding_idx=padding_idx)
        # define parameters with initial weights: 0 for the padding and uniformly sampled weights for the vocab
        self.ivectors.weight = nn.Parameter(t.cat([t.zeros(1, self.embedding_size), FT(self.vocab_size - 1, self.embedding_size).uniform_(-0.5 / self.embedding_size, 0.5 / self.embedding_size)]))
        self.ovectors.weight = nn.Parameter(t.cat([t.zeros(1, self.embedding_size), FT(self.vocab_size - 1, self.embedding_size).uniform_(-0.5 / self.embedding_size, 0.5 / self.embedding_size)]))
        # indicates that we do not exclude this as it is the main part of the graph - this should be treated as parameters
        self.ivectors.weight.requires_grad = True
        self.ovectors.weight.requires_grad = True

    def forward(self, data):
        return self.forward_i(data)

    def forward_i(self, data):
        v = LT(data)
        v = v.cuda() if self.ivectors.weight.is_cuda else v
        return self.ivectors(v)

    def forward_o(self, data):
        v = LT(data)
        v = v.cuda() if self.ovectors.weight.is_cuda else v
        return self.ovectors(v)


class SGNS(nn.Module):

    def __init__(self, embedding, vocab_size=20000, n_negs=20, weights=None):
        super(SGNS, self).__init__()
        self.embedding = embedding
        self.vocab_size = vocab_size
        self.n_negs = n_negs
        self.weights = None
        if weights is not None:
            wf = np.power(weights, 0.75)
            wf = wf / wf.sum()
            self.weights = FT(wf)

    def forward(self, iword, owords):
        batch_size = iword.size()[0]
        context_size = owords.size()[1]
        if self.weights is not None:
            nwords = t.multinomial(self.weights, batch_size * context_size * self.n_negs, replacement=True).view(batch_size, -1)
        else:
            nwords = FT(batch_size, context_size * self.n_negs).uniform_(0, self.vocab_size - 1).long()
        ivectors = self.embedding.forward_i(iword).unsqueeze(2)
        ovectors = self.embedding.forward_o(owords)
        nvectors = self.embedding.forward_o(nwords).neg()
##-------------- this is the line we need to modify: add the euclidean distance as a regularisation term
##-------------- we will also consider the consine similarity
        oloss = t.bmm(ovectors, ivectors).squeeze().sigmoid().log().mean(1)
        nloss = t.bmm(nvectors, ivectors).squeeze().sigmoid().log().view(-1, context_size, self.n_negs).sum(2).mean(1)
        return(-(oloss + nloss).mean())

### code for training

In [3]:
import os
import pickle
import random
import argparse
import torch as t
import numpy as np

from tqdm import tqdm
from torch.optim import Adam
from torch.optim import SGD
from torch.utils.data import Dataset, DataLoader
#from model import Word2Vec, SGNS


def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--name', type=str, default='sgns', help="model name")
    parser.add_argument('--data_dir', type=str, default='./data/', help="data directory path")
    parser.add_argument('--save_dir', type=str, default='./pts/', help="model directory path")
    parser.add_argument('--e_dim', type=int, default=300, help="embedding dimension")
    parser.add_argument('--n_negs', type=int, default=20, help="number of negative samples")
    parser.add_argument('--epoch', type=int, default=100, help="number of epochs")
    parser.add_argument('--mb', type=int, default=4096, help="mini-batch size")
    parser.add_argument('--ss_t', type=float, default=1e-5, help="subsample threshold")
    parser.add_argument('--conti', action='store_true', help="continue learning")
    parser.add_argument('--weights', action='store_true', help="use weights for negative sampling")
    parser.add_argument('--cuda', action='store_true', help="use CUDA")
    return parser.parse_args()


class PermutedSubsampledCorpus(Dataset):

    def __init__(self, datapath, ws=None):
        data = pickle.load(open(datapath, 'rb'))
        if ws is not None:
            self.data = []
            for iword, owords in data:
                if random.random() > ws[iword]:
                    self.data.append((iword, owords))
        else:
            self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        iword, owords = self.data[idx]
        return iword, np.array(owords)


def train(args):
    idx2word = pickle.load(open(os.path.join(args.data_dir, 'idx2word.dat'), 'rb'))
    wc = pickle.load(open(os.path.join(args.data_dir, 'wc.dat'), 'rb'))
    wf = np.array([wc[word] for word in idx2word])
    wf = wf / wf.sum()
    ws = 1 - np.sqrt(args.ss_t / wf)
    ws = np.clip(ws, 0, 1)
    vocab_size = len(idx2word)
    weights = wf if args.weights else None
    if not os.path.isdir(args.save_dir):
        os.mkdir(args.save_dir)
    model = Word2Vec(vocab_size=vocab_size, embedding_size=args.e_dim)
    modelpath = os.path.join(args.save_dir, '{}.pt'.format(args.name))
    sgns = SGNS(embedding=model, vocab_size=vocab_size, n_negs=args.n_negs, weights=weights)
    if os.path.isfile(modelpath) and args.conti:
        sgns.load_state_dict(t.load(modelpath))
    if args.cuda:
        sgns = sgns.cuda()
    optim = Adam(sgns.parameters())
    optimpath = os.path.join(args.save_dir, '{}.optim.pt'.format(args.name))
    if os.path.isfile(optimpath) and args.conti:
        optim.load_state_dict(t.load(optimpath))
    for epoch in range(1, args.epoch + 1):
        dataset = PermutedSubsampledCorpus(os.path.join(args.data_dir, 'train.dat'))
        dataloader = DataLoader(dataset, batch_size=args.mb, shuffle=True)
        total_batches = int(np.ceil(len(dataset) / args.mb))
        pbar = tqdm(dataloader)
        pbar.set_description("[Epoch {}]".format(epoch))
        for iword, owords in pbar:
            loss = sgns(iword, owords)
            optim.zero_grad()
            loss.backward()
            optim.step()
            pbar.set_postfix(loss=loss.item())
    idx2vec = model.ivectors.weight.data.cpu().numpy()
    pickle.dump(idx2vec, open(os.path.join(args.data_dir, 'idx2vec.dat'), 'wb'))
    t.save(sgns.state_dict(), os.path.join(args.save_dir, '{}.pt'.format(args.name)))
    t.save(optim.state_dict(), os.path.join(args.save_dir, '{}.optim.pt'.format(args.name)))


#if __name__ == '__main__':
 #   train(parse_args())

### We'll go through this line by line

In [4]:
import os
import pickle
import random

import argparse
import torch as t
import numpy as np

from tqdm import tqdm
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
#from model import Word2Vec, SGNS

In [5]:
# loading in idx2word and word counts and calculating the normalised frequency for words in the idx

with(open(os.path.join(data_path, 'idx2word.dat'), 'rb')) as handle:
    idx2word = pickle.load(handle)
with(open(os.path.join(data_path, 'wc.dat'), 'rb')) as handle:
    wc = pickle.load(handle)
wf = np.array([wc[word] for word in idx2word])
wf = wf / wf.sum()

In [6]:
# calculating the subsampling proba and obtaining vocab size

ss_t = 0.00001

ws = 1 - np.sqrt(ss_t / wf)
ws = np.clip(ws, 0, 1)
vocab_size = len(idx2word)

In [14]:
# do we wanna use weights for negsampling?
use_weights = True
weights = wf if use_weights else None

In [15]:
save_dir = data_path+'res/'
if not os.path.isdir(save_dir):
    os.mkdir(save_dir)

In [16]:
# call and instantiate the model class
emb_dim = 300
model_name = 'period0'

model = Word2Vec(vocab_size=vocab_size, embedding_size=emb_dim)
modelpath = os.path.join(save_dir, '{}.pt'.format(model_name))

In [17]:
# call the SGNS model and instantiate
n_negs = 100

sgns = SGNS(embedding=model, vocab_size=vocab_size, n_negs=n_negs, weights=weights)

In [18]:
# do we wanna pick up from a previous learning?
# do we wanna use CUDA

conti = False
cuda = True

if os.path.isfile(modelpath) and conti:
    sgns.load_state_dict(t.load(modelpath))
if cuda:
    sgns = sgns.cuda()

In [19]:
# call the optimiser, we use Adam, can also use SGD
optim = Adam(sgns.parameters(), lr=2e-3)
optimpath = os.path.join(save_dir, '{}.optim.pt'.format(model_name))
if os.path.isfile(optimpath) and conti:
    optim.load_state_dict(t.load(optimpath))

In [20]:
# train!
n_epoch = 8
mb = 1000

for epoch in range(1, n_epoch + 1):
    dataset = PermutedSubsampledCorpus(os.path.join(data_path, 'train.dat'), ws)
    dataloader = DataLoader(dataset, batch_size=mb, shuffle=True)
    total_batches = int(np.ceil(len(dataset) / mb))
    pbar = tqdm(dataloader)
    pbar.set_description("[Epoch {}]".format(epoch))
    for iword, owords in pbar:
        loss = sgns(iword, owords)
        #grad = sgns(iword, owords)[1]
        #print(grad)
        optim.zero_grad()
        loss.backward()
        optim.step()
        pbar.set_postfix(loss=loss.item())

[Epoch 8]: 100%|██████████| 717/717 [04:31<00:00,  2.64it/s, loss=17.2]


In [21]:
idx2vec = model.ivectors.weight.data.cpu().numpy()
#pickle.dump(idx2vec, open(data_path+'idx2vec.dat'), 'wb')
t.save(sgns.state_dict(), os.path.join(save_dir, '{}.pt'.format(model_name)))
t.save(optim.state_dict(), os.path.join(save_dir, '{}.optim.pt'.format(model_name)))

In [22]:
model.ivectors.weight.data.cpu().numpy()

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.05689228,  0.15685235,  0.07270526, ...,  0.15389794,
         0.09007584,  0.0774914 ],
       [ 0.00632227,  0.01002212,  0.09341568, ...,  0.10295281,
         0.15408444,  0.09707116],
       ..., 
       [ 0.03852747,  0.06537829,  0.04834054, ...,  0.07518096,
         0.08379278,  0.0544805 ],
       [ 0.03027225,  0.02561502, -0.0026253 , ...,  0.0233258 ,
         0.02598815,  0.05608466],
       [ 0.0625817 ,  0.057478  ,  0.05790081, ...,  0.00474848,
        -0.03421123,  0.05335202]], dtype=float32)

In [23]:
vector3 = model.ivectors.weight.data.cpu().numpy()[3]
vector3

array([ 0.05789586,  0.04476868,  0.09167903, -0.17152733,  0.10329971,
        0.03014417,  0.01219846, -0.13819744, -0.00565925,  0.03899888,
        0.03732952, -0.0898168 , -0.07575436,  0.02051679, -0.02981773,
       -0.07936405,  0.00382746, -0.01618091, -0.08324953, -0.1328506 ,
        0.10098054, -0.08548413,  0.07240234,  0.02332029,  0.05415119,
       -0.06823785, -0.05450507,  0.09783662, -0.01419741,  0.00150108,
       -0.00262661, -0.04425149, -0.13105522, -0.00611185, -0.02356478,
        0.02706797, -0.09736105,  0.04132115, -0.03511995,  0.00675131,
        0.02178738,  0.01402447, -0.11562996, -0.01941235, -0.0577183 ,
       -0.03773048, -0.00415369,  0.01645307,  0.05912367,  0.03455342,
        0.0583756 , -0.0349873 , -0.00496768, -0.09155397,  0.06138192,
        0.03567986, -0.13077518,  0.01166302,  0.13653545, -0.11633229,
        0.05879457,  0.02754627,  0.03053075, -0.00725776,  0.19709015,
        0.04714454, -0.09065689,  0.02722152,  0.0851267 , -0.04

In [24]:
with open(data_path + 'word2idx.dat', 'rb') as handle:
    word2idx = pickle.load(handle)

In [25]:
word0, word1 = 'shoe','great'
if word0 in idx2word and word1 in idx2word:
    comp_idx = word2idx[word0], word2idx[word1]
    print(comp_idx)
else:
    print('not found')

(11820, 337)


In [26]:
array0, array1 = model.ivectors.weight.data.cpu().numpy()[comp_idx[0]],model.ivectors.weight.data.cpu().numpy()[comp_idx[1]]

In [27]:
from scipy import spatial

result = 1 - spatial.distance.cosine(array0, array1)
print(result)

0.501607239246


In [36]:
res = [1 - spatial.distance.cosine(model.ivectors.weight.data.cpu().numpy()[3], model.ivectors.weight.data.cpu().numpy()[i]) for i in tqdm(range(1, vocab_size))]


  4%|▍         | 827/19999 [00:17<06:46, 47.21it/s]


  8%|▊         | 1650/19999 [00:34<06:28, 47.24it/s]


 12%|█▏        | 2464/19999 [00:52<06:12, 47.12it/s]


 16%|█▌        | 3237/19999 [01:09<06:01, 46.39it/s]


 20%|██        | 4027/19999 [01:27<05:46, 46.14it/s]


...

 87%|████████▋ | 17434/19999 [06:19<00:55, 45.89it/s]


 91%|█████████ | 18196/19999 [06:36<00:39, 45.84it/s]


 95%|█████████▍| 18939/19999 [06:53<00:23, 45.77it/s]


 99%|█████████▊| 19722/19999 [07:10<00:06, 45.78it/s]


100%|██████████| 19999/19999 [07:16<00:00, 45.77it/s]


In [68]:
np.mean(res), np.std(res)

(0.41633244453847357, 0.077514127213931847)

Here we try to test adding regularisation

In [29]:
import numpy as np
import torch as t
import torch.nn as nn

from torch import LongTensor as LT
from torch import FloatTensor as FT

 
class Bundler(nn.Module):

    def forward(self, data):
        raise NotImplementedError

    def forward_i(self, data):
        raise NotImplementedError

    def forward_o(self, data):
        raise NotImplementedError


class Word2Vec(Bundler):

    def __init__(self, vocab_size=20000, embedding_size=300, padding_idx=0, vector_3 = None):
        # this instantiate the parent class
        super(Word2Vec, self).__init__()
        self.vocab_size = vocab_size
        self.embedding_size = embedding_size
        # define embeddings for the target word matrix and the context word matrix
        self.ivectors = nn.Embedding(self.vocab_size, self.embedding_size, padding_idx=padding_idx)
        self.ovectors = nn.Embedding(self.vocab_size, self.embedding_size, padding_idx=padding_idx)
        #self.vector_3 = nn.Embedding(self.vocab_size, 1, padding_idx=None)
        # define parameters with initial weights: 0 for the padding and uniformly sampled weights for the vocab
        self.ivectors.weight = nn.Parameter(t.cat([t.zeros(1, self.embedding_size), FT(self.vocab_size - 1, self.embedding_size).uniform_(-0.5 / self.embedding_size, 0.5 / self.embedding_size)]))
        self.ovectors.weight = nn.Parameter(t.cat([t.zeros(1, self.embedding_size), FT(self.vocab_size - 1, self.embedding_size).uniform_(-0.5 / self.embedding_size, 0.5 / self.embedding_size)]))
        #self.vector_3.weight = FT(vector_3)
        # indicates that we do not exclude this as it is the main part of the graph - this should be treated as parameters
        self.ivectors.weight.requires_grad = True
        self.ovectors.weight.requires_grad = True
        #self.vector_3.weight.requires_grad = False

    def forward(self, data):
        return self.forward_i(data)

    def forward_i(self, data):
        if data is not None:
            v = LT(data)
            v = v.cuda() if self.ivectors.weight.is_cuda else v
            return self.ivectors(v)
        else:
            return(None)

    def forward_o(self, data):
        v = LT(data)
        v = v.cuda() if self.ovectors.weight.is_cuda else v
        return self.ovectors(v)

class SGNS(nn.Module):

    def __init__(self, embedding, vocab_size=20000, n_negs=20, weights=None, test_tensor = None, vector3 = None):
        super(SGNS, self).__init__()
        self.embedding = embedding
        self.vocab_size = vocab_size
        self.n_negs = n_negs
        self.weights = None
        self.vector3 = FT(vector3).cuda()
        self.vector3.requires_grad = False
        if weights is not None:
            wf = np.power(weights, 0.75)
            wf = wf / wf.sum()
            self.weights = FT(wf)

    def forward(self, iword, owords, rwords):
        batch_size = iword.size()[0]
        context_size = owords.size()[1]
        if self.weights is not None:
            nwords = t.multinomial(self.weights, batch_size * context_size * self.n_negs, replacement=True).view(batch_size, -1)
        else:
            nwords = FT(batch_size, context_size * self.n_negs).uniform_(0, self.vocab_size - 1).long()
        ivectors = self.embedding.forward_i(iword).unsqueeze(2)
        #print('ivectors', ivectors.shape)
        ovectors = self.embedding.forward_o(owords)
        #print('ovectors', ovectors.shape)
        nvectors = self.embedding.forward_o(nwords).neg()
        #print(rvectors.shape)
##-------------- this is the line we need to modify: add the euclidean distance as a regularisation term
##-------------- we will also consider the consine similarity
        oloss = t.bmm(ovectors, ivectors).squeeze().sigmoid().log().mean(1)
        #print(oloss.shape)
        nloss = t.bmm(nvectors, ivectors).squeeze().sigmoid().log().view(-1, context_size, self.n_negs).sum(2).mean(1)
        #print(t.bmm(nvectors, ivectors).squeeze().sigmoid().log().view(-1, context_size, self.n_negs).sum(2).mean(1).shape)
        #rloss = 3*((rvectors.squeeze() - self.vector3)**2).sum()
        if self.embedding.forward_i(rwords) is not None:
            MSE_loss_fn = nn.MSELoss(reduction='sum')
            rvectors = self.embedding.forward_i(rwords)
            rloss = MSE_loss_fn(rvectors.squeeze(), self.vector3)
            return(-(oloss + nloss).mean() + 10*rloss)
        else:
            return(-(oloss + nloss).mean())
        #print((-(oloss + nloss).mean()).shape)

In [30]:
emb_dim = 300
model_name = 'period0_1'

model_1 = Word2Vec(vocab_size=vocab_size, embedding_size=emb_dim)
modelpath = os.path.join(save_dir, '{}.pt'.format(model_name))

n_negs = 100

sgns_1 = SGNS(embedding=model_1, vocab_size=vocab_size, n_negs=n_negs, weights=weights, vector3=vector3)

In [31]:
sgns_1 = sgns_1.cuda()

In [32]:
optim_1 = Adam(sgns_1.parameters())
optimpath = os.path.join(save_dir, '{}.optim.pt'.format(model_name))
if os.path.isfile(optimpath) and conti:
    optim_1.load_state_dict(t.load(optimpath))

In [33]:
n_epoch = 10
mb = 1000

for epoch in range(1, n_epoch + 1):
    dataset = PermutedSubsampledCorpus(os.path.join(data_path, 'train.dat'), ws)
    dataloader = DataLoader(dataset, batch_size=mb, shuffle=True)
    total_batches = int(np.ceil(len(dataset) / mb))
    pbar = tqdm(dataloader)
    pbar.set_description("[Epoch {}]".format(epoch))
    for iword, owords in pbar:
        rwords = LT([3]) if 3 in iword else None
        loss = sgns_1(iword, owords, rwords)
        #grad = sgns_1(iword, owords, rwords)[1]
        optim_1.zero_grad()
        loss.backward()
        #print(grad)
        #print(list(model_1.parameters()))
        optim_1.step()
        pbar.set_postfix(loss=loss.item())

[Epoch 1]:   8%|▊         | 54/716 [00:21<04:25,  2.49it/s, loss=65]


[Epoch 1]:  15%|█▌        | 109/716 [00:42<03:57,  2.55it/s, loss=36.6]


[Epoch 1]:  23%|██▎       | 164/716 [01:03<03:33,  2.59it/s, loss=24.1]


[Epoch 1]:  30%|███       | 218/716 [01:24<03:13,  2.58it/s, loss=18.4]


[Epoch 1]:  38%|███▊      | 273/716 [01:45<02:50,  2.60it/s, loss=17]


...

[Epoch 10]:  74%|███████▍  | 533/716 [03:23<01:10,  2.61it/s, loss=13.9]


[Epoch 10]:  82%|████████▏ | 586/716 [03:44<00:49,  2.61it/s, loss=12.7]


[Epoch 10]:  89%|████████▉ | 640/716 [04:04<00:29,  2.61it/s, loss=13.8]


[Epoch 10]:  97%|█████████▋| 693/716 [04:25<00:08,  2.61it/s, loss=13.6]


[Epoch 10]: 100%|██████████| 716/716 [04:33<00:00,  2.61it/s, loss=14.1]


In [34]:
model_1.ivectors.weight.data.cpu().numpy()[3]

array([ 0.05816619,  0.04498733,  0.09160124, -0.17151251,  0.10322422,
        0.03000014,  0.01200111, -0.13853897, -0.00585345,  0.03884761,
        0.0371792 , -0.09004281, -0.07565466,  0.02033259, -0.02968116,
       -0.07956991,  0.00401943, -0.01603008, -0.08317608, -0.13277908,
        0.10089583, -0.08572169,  0.0722882 ,  0.0231715 ,  0.05441654,
       -0.06812048, -0.05478003,  0.09775572, -0.01404165,  0.00129854,
       -0.00281095, -0.04445899, -0.13130639, -0.00629688, -0.02344881,
        0.02692058, -0.09760157,  0.04150186, -0.03530991,  0.00656995,
        0.02200015,  0.01385296, -0.11589619, -0.01922443, -0.05797816,
       -0.03792982, -0.0039581 ,  0.01625879,  0.05939846,  0.03436416,
        0.05823668, -0.03517651, -0.00515196, -0.09179679,  0.06125971,
        0.03553877, -0.13070497,  0.01148216,  0.13685359, -0.11627848,
        0.05867105,  0.02773466,  0.03072521, -0.00707116,  0.19710493,
        0.04737592, -0.09089682,  0.02742238,  0.08533072, -0.04

In [35]:
vector3

array([ 0.05789586,  0.04476868,  0.09167903, -0.17152733,  0.10329971,
        0.03014417,  0.01219846, -0.13819744, -0.00565925,  0.03899888,
        0.03732952, -0.0898168 , -0.07575436,  0.02051679, -0.02981773,
       -0.07936405,  0.00382746, -0.01618091, -0.08324953, -0.1328506 ,
        0.10098054, -0.08548413,  0.07240234,  0.02332029,  0.05415119,
       -0.06823785, -0.05450507,  0.09783662, -0.01419741,  0.00150108,
       -0.00262661, -0.04425149, -0.13105522, -0.00611185, -0.02356478,
        0.02706797, -0.09736105,  0.04132115, -0.03511995,  0.00675131,
        0.02178738,  0.01402447, -0.11562996, -0.01941235, -0.0577183 ,
       -0.03773048, -0.00415369,  0.01645307,  0.05912367,  0.03455342,
        0.0583756 , -0.0349873 , -0.00496768, -0.09155397,  0.06138192,
        0.03567986, -0.13077518,  0.01166302,  0.13653545, -0.11633229,
        0.05879457,  0.02754627,  0.03053075, -0.00725776,  0.19709015,
        0.04714454, -0.09065689,  0.02722152,  0.0851267 , -0.04

In [184]:
sum([0 == tens for tens in LT([0,1,1,2,5,6,3]).numpy()])

1

In [191]:
LT(vector3).shape

torch.Size([300])

In [210]:
print(LT(np.array([[2,5,8],[4,5,77]])).shape, LT([1,2,3]).shape)

torch.Size([2, 3]) torch.Size([3])


In [259]:
sgns_1(LT([1,2,3]), LT(np.array([[2,5,8],[4,5,77]]).T), None)

ivectors torch.Size([3, 300, 1])
ovectors torch.Size([3, 2, 300])
nvectors torch.Size([3, 200, 300])
torch.Size([3])
torch.Size([3])
torch.Size([])


tensor(70.0079, device='cuda:0', grad_fn=<NegBackward>)

In [196]:
n_epoch = 10
mb = 1000

for epoch in range(1, n_epoch + 1):
    dataset = PermutedSubsampledCorpus(os.path.join(data_path, 'train.dat'), ws)
    dataloader = DataLoader(dataset, batch_size=mb, shuffle=True)
    total_batches = int(np.ceil(len(dataset) / mb))
    pbar = tqdm(dataloader)
    pbar.set_description("[Epoch {}]".format(epoch))
    for iword, owords in pbar:
        print(iword.shape)
        print(owords.shape)
        print('----')


[Epoch 1]:   2%|▏         | 14/715 [00:00<00:23, 30.14it/s]

torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1

[Epoch 1]:   6%|▋         | 46/715 [00:00<00:09, 68.31it/s]

torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1

[Epoch 1]:  11%|█         | 79/715 [00:00<00:07, 89.40it/s]

...

torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1

[Epoch 10]:  96%|█████████▌| 686/715 [00:04<00:00, 142.77it/s]

torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1

[Epoch 10]: 100%|██████████| 715/715 [00:04<00:00, 143.07it/s]

torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([323])
torch.Size([323, 10])
----


In [37]:
res1 = [1 - spatial.distance.cosine(model_1.ivectors.weight.data.cpu().numpy()[3], model_1.ivectors.weight.data.cpu().numpy()[i]) for i in tqdm(range(1, vocab_size))]


  4%|▍         | 790/19999 [00:17<07:04, 45.24it/s]


  8%|▊         | 1591/19999 [00:34<06:43, 45.65it/s]


 12%|█▏        | 2412/19999 [00:52<06:21, 46.07it/s]


 16%|█▌        | 3199/19999 [01:09<06:07, 45.77it/s]


 20%|██        | 4012/19999 [01:27<05:48, 45.94it/s]


...

 87%|████████▋ | 17319/19999 [06:21<00:58, 45.43it/s]


 91%|█████████ | 18111/19999 [06:38<00:41, 45.47it/s]


 95%|█████████▍| 18916/19999 [06:55<00:23, 45.55it/s]


 98%|█████████▊| 19662/19999 [07:12<00:07, 45.45it/s]


100%|██████████| 19999/19999 [07:20<00:00, 45.44it/s]

In [38]:
spatial.distance.cosine(res, res1)

0.94664735631031804